# CIFAR-10 Challange - 김영인

1. resnet152
2. \+ Data Augmentation(RandomResizedCrop, RandomHorizontalFlip)
3. \+ Data Normalize
4. \+ regularization(dropout, weight decay)
5. \+ Optimizer(momentum, scheduler)


In [1]:
import random

import torch
import numpy as np
from matplotlib import pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets

random_seed = 4332
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

device0 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device1 = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

device = device1
print(f"device: {device}") if torch.cuda.is_available() else print("device: cpu")

device: cuda:1


## Hyper parameter Setting 

In [2]:
learning_rate = 0.001
training_epochs = 100
batch_size = 16
momentum = 0.9
scheduler_step = 1
scheduler_gamma = 0.99

## Load & Preprocess Data

In [3]:
transform_train = transforms.Compose(
    [transforms.RandomResizedCrop(224), # data augmentation, 224: image size, ImageNet pretrained model에 맞추기 위해서 224 size로 설정
     transforms.RandomHorizontalFlip(), # data augmentation, 좌우로 대칭
     transforms.ToTensor(), # numpy array를 pytorch tensor로 바꿔주는 역할
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # dataset의 mean, std를 이용해서 -1~1 로 normalize
    ])

transform_test = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True,
                                      download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=4,
                                        shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## Load and Define pretrained model and fine-tuning

In [4]:
class ResNet152(nn.Module):
    def __init__ (self):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 10)
        
    def forward(self, x):
        x = self.resnet(x) # batch_size * 10
        return x

## Load Model

In [5]:
net = ResNet152()
net = net.to(device)

criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                      step_size = scheduler_step,
                                      gamma = scheduler_gamma)

## Train

In [6]:
net.train()

for epoch in range(training_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()

        outputs = net(inputs) 
        loss =  criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
            
        if i % 750 == 0:
            class_correct = np.zeros(10)
            class_total = np.zeros(10)

            net.eval()

            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1
            print('==================')
            print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))
            print('==================')
            
            net.train()
        
print('Finished Training')

Mean Accuracy :  8.71
[1,   100] loss: 1.856
[1,   200] loss: 1.151
[1,   300] loss: 0.937
[1,   400] loss: 0.914
[1,   500] loss: 0.906
[1,   600] loss: 0.810
[1,   700] loss: 0.797
Mean Accuracy :  77.27
[1,   800] loss: 0.765
[1,   900] loss: 0.784
[1,  1000] loss: 0.763
[1,  1100] loss: 0.701
[1,  1200] loss: 0.740
[1,  1300] loss: 0.671
[1,  1400] loss: 0.659
[1,  1500] loss: 0.662
Mean Accuracy :  79.29
[1,  1600] loss: 0.722
[1,  1700] loss: 0.654
[1,  1800] loss: 0.642
[1,  1900] loss: 0.658
[1,  2000] loss: 0.635
[1,  2100] loss: 0.607
[1,  2200] loss: 0.654
Mean Accuracy :  81.38
[1,  2300] loss: 0.684
[1,  2400] loss: 0.633
[1,  2500] loss: 0.632
[1,  2600] loss: 0.617
[1,  2700] loss: 0.587
[1,  2800] loss: 0.596
[1,  2900] loss: 0.587
[1,  3000] loss: 0.606
Mean Accuracy :  80.87
[1,  3100] loss: 0.618
Mean Accuracy :  83.22
[2,   100] loss: 0.497
[2,   200] loss: 0.558
[2,   300] loss: 0.606
[2,   400] loss: 0.501
[2,   500] loss: 0.540
[2,   600] loss: 0.529
[2,   700] l

[9,   100] loss: 0.319
[9,   200] loss: 0.354
[9,   300] loss: 0.362
[9,   400] loss: 0.339
[9,   500] loss: 0.319
[9,   600] loss: 0.364
[9,   700] loss: 0.329
Mean Accuracy :  87.1
[9,   800] loss: 0.379
[9,   900] loss: 0.309
[9,  1000] loss: 0.342
[9,  1100] loss: 0.333
[9,  1200] loss: 0.299
[9,  1300] loss: 0.316
[9,  1400] loss: 0.285
[9,  1500] loss: 0.316
Mean Accuracy :  87.13
[9,  1600] loss: 0.330
[9,  1700] loss: 0.304
[9,  1800] loss: 0.318
[9,  1900] loss: 0.341
[9,  2000] loss: 0.346
[9,  2100] loss: 0.337
[9,  2200] loss: 0.355
Mean Accuracy :  87.87
[9,  2300] loss: 0.309
[9,  2400] loss: 0.349
[9,  2500] loss: 0.346
[9,  2600] loss: 0.366
[9,  2700] loss: 0.316
[9,  2800] loss: 0.339
[9,  2900] loss: 0.330
[9,  3000] loss: 0.316
Mean Accuracy :  87.13
[9,  3100] loss: 0.321
Mean Accuracy :  87.99
[10,   100] loss: 0.315
[10,   200] loss: 0.329
[10,   300] loss: 0.325
[10,   400] loss: 0.303
[10,   500] loss: 0.365
[10,   600] loss: 0.318
[10,   700] loss: 0.288
Mean 

Mean Accuracy :  89.06
[16,  3100] loss: 0.273
Mean Accuracy :  88.87
[17,   100] loss: 0.252
[17,   200] loss: 0.279
[17,   300] loss: 0.273
[17,   400] loss: 0.252
[17,   500] loss: 0.280
[17,   600] loss: 0.269
[17,   700] loss: 0.259
Mean Accuracy :  88.65
[17,   800] loss: 0.263
[17,   900] loss: 0.259
[17,  1000] loss: 0.258
[17,  1100] loss: 0.271
[17,  1200] loss: 0.273
[17,  1300] loss: 0.277
[17,  1400] loss: 0.249
[17,  1500] loss: 0.281
Mean Accuracy :  89.0
[17,  1600] loss: 0.272
[17,  1700] loss: 0.251
[17,  1800] loss: 0.301
[17,  1900] loss: 0.249
[17,  2000] loss: 0.242
[17,  2100] loss: 0.259
[17,  2200] loss: 0.258
Mean Accuracy :  88.33
[17,  2300] loss: 0.285
[17,  2400] loss: 0.274
[17,  2500] loss: 0.253
[17,  2600] loss: 0.298
[17,  2700] loss: 0.263
[17,  2800] loss: 0.305
[17,  2900] loss: 0.257
[17,  3000] loss: 0.289
Mean Accuracy :  88.64
[17,  3100] loss: 0.260
Mean Accuracy :  88.3
[18,   100] loss: 0.257
[18,   200] loss: 0.244
[18,   300] loss: 0.281
[

[24,  2300] loss: 0.292
[24,  2400] loss: 0.234
[24,  2500] loss: 0.226
[24,  2600] loss: 0.272
[24,  2700] loss: 0.243
[24,  2800] loss: 0.223
[24,  2900] loss: 0.304
[24,  3000] loss: 0.258
Mean Accuracy :  89.74
[24,  3100] loss: 0.253
Mean Accuracy :  89.47
[25,   100] loss: 0.245
[25,   200] loss: 0.273
[25,   300] loss: 0.230
[25,   400] loss: 0.258
[25,   500] loss: 0.243
[25,   600] loss: 0.226
[25,   700] loss: 0.242
Mean Accuracy :  89.37
[25,   800] loss: 0.201
[25,   900] loss: 0.261
[25,  1000] loss: 0.259
[25,  1100] loss: 0.227
[25,  1200] loss: 0.248
[25,  1300] loss: 0.251
[25,  1400] loss: 0.266
[25,  1500] loss: 0.200
Mean Accuracy :  88.8
[25,  1600] loss: 0.261
[25,  1700] loss: 0.224
[25,  1800] loss: 0.236
[25,  1900] loss: 0.221
[25,  2000] loss: 0.233
[25,  2100] loss: 0.221
[25,  2200] loss: 0.266
Mean Accuracy :  89.18
[25,  2300] loss: 0.257
[25,  2400] loss: 0.223
[25,  2500] loss: 0.245
[25,  2600] loss: 0.268
[25,  2700] loss: 0.243
[25,  2800] loss: 0.27

[32,  1800] loss: 0.232
[32,  1900] loss: 0.218
[32,  2000] loss: 0.204
[32,  2100] loss: 0.214
[32,  2200] loss: 0.224
Mean Accuracy :  89.68
[32,  2300] loss: 0.241
[32,  2400] loss: 0.243
[32,  2500] loss: 0.261
[32,  2600] loss: 0.216
[32,  2700] loss: 0.230
[32,  2800] loss: 0.237
[32,  2900] loss: 0.245
[32,  3000] loss: 0.219
Mean Accuracy :  89.94
[32,  3100] loss: 0.246
Mean Accuracy :  89.77
[33,   100] loss: 0.237
[33,   200] loss: 0.224
[33,   300] loss: 0.222
[33,   400] loss: 0.225
[33,   500] loss: 0.219
[33,   600] loss: 0.211
[33,   700] loss: 0.217
Mean Accuracy :  89.58
[33,   800] loss: 0.241
[33,   900] loss: 0.220
[33,  1000] loss: 0.201
[33,  1100] loss: 0.210
[33,  1200] loss: 0.208
[33,  1300] loss: 0.229
[33,  1400] loss: 0.205
[33,  1500] loss: 0.205
Mean Accuracy :  89.56
[33,  1600] loss: 0.225
[33,  1700] loss: 0.220
[33,  1800] loss: 0.255
[33,  1900] loss: 0.230
[33,  2000] loss: 0.210
[33,  2100] loss: 0.230
[33,  2200] loss: 0.218
Mean Accuracy :  90.1

...

[63,  3000] loss: 0.208
Mean Accuracy :  90.77
[63,  3100] loss: 0.190
Mean Accuracy :  90.89
[64,   100] loss: 0.175
[64,   200] loss: 0.171
[64,   300] loss: 0.196
[64,   400] loss: 0.191
[64,   500] loss: 0.184
[64,   600] loss: 0.182
[64,   700] loss: 0.139
Mean Accuracy :  90.43
[64,   800] loss: 0.172
[64,   900] loss: 0.180
[64,  1000] loss: 0.172
[64,  1100] loss: 0.180
[64,  1200] loss: 0.196
[64,  1300] loss: 0.176
[64,  1400] loss: 0.186
[64,  1500] loss: 0.161
Mean Accuracy :  91.04
[64,  1600] loss: 0.189
[64,  1700] loss: 0.182
[64,  1800] loss: 0.175
[64,  1900] loss: 0.175
[64,  2000] loss: 0.181
[64,  2100] loss: 0.169
[64,  2200] loss: 0.177
Mean Accuracy :  90.06
[64,  2300] loss: 0.183
[64,  2400] loss: 0.167
[64,  2500] loss: 0.203
[64,  2600] loss: 0.166
[64,  2700] loss: 0.165
[64,  2800] loss: 0.191
[64,  2900] loss: 0.181
[64,  3000] loss: 0.182
Mean Accuracy :  90.34
[64,  3100] loss: 0.187
Mean Accuracy :  90.65
[65,   100] loss: 0.180
[65,   200] loss: 0.186

[71,  2300] loss: 0.153
[71,  2400] loss: 0.158
[71,  2500] loss: 0.179
[71,  2600] loss: 0.162
[71,  2700] loss: 0.191
[71,  2800] loss: 0.179
[71,  2900] loss: 0.177
[71,  3000] loss: 0.181
Mean Accuracy :  90.41
[71,  3100] loss: 0.167
Mean Accuracy :  90.24
[72,   100] loss: 0.179
[72,   200] loss: 0.164
[72,   300] loss: 0.148
[72,   400] loss: 0.159
[72,   500] loss: 0.182
[72,   600] loss: 0.166
[72,   700] loss: 0.184
Mean Accuracy :  91.18
[72,   800] loss: 0.194
[72,   900] loss: 0.161
[72,  1000] loss: 0.174
[72,  1100] loss: 0.192
[72,  1200] loss: 0.174
[72,  1300] loss: 0.198
[72,  1400] loss: 0.156
[72,  1500] loss: 0.193
Mean Accuracy :  90.81
[72,  1600] loss: 0.185
[72,  1700] loss: 0.193
[72,  1800] loss: 0.169
[72,  1900] loss: 0.206
[72,  2000] loss: 0.159
[72,  2100] loss: 0.162
[72,  2200] loss: 0.175
Mean Accuracy :  91.47
[72,  2300] loss: 0.169
[72,  2400] loss: 0.201
[72,  2500] loss: 0.180
[72,  2600] loss: 0.179
[72,  2700] loss: 0.172
[72,  2800] loss: 0.1

[79,  1800] loss: 0.159
[79,  1900] loss: 0.179
[79,  2000] loss: 0.182
[79,  2100] loss: 0.177
[79,  2200] loss: 0.170
Mean Accuracy :  90.35
[79,  2300] loss: 0.188
[79,  2400] loss: 0.180
[79,  2500] loss: 0.156
[79,  2600] loss: 0.176
[79,  2700] loss: 0.179
[79,  2800] loss: 0.171
[79,  2900] loss: 0.178
[79,  3000] loss: 0.146
Mean Accuracy :  90.81
[79,  3100] loss: 0.189
Mean Accuracy :  91.04
[80,   100] loss: 0.166
[80,   200] loss: 0.168
[80,   300] loss: 0.161
[80,   400] loss: 0.149
[80,   500] loss: 0.167
[80,   600] loss: 0.165
[80,   700] loss: 0.170
Mean Accuracy :  90.63
[80,   800] loss: 0.134
[80,   900] loss: 0.179
[80,  1000] loss: 0.158
[80,  1100] loss: 0.166
[80,  1200] loss: 0.205
[80,  1300] loss: 0.181
[80,  1400] loss: 0.166
[80,  1500] loss: 0.182
Mean Accuracy :  90.44
[80,  1600] loss: 0.175
[80,  1700] loss: 0.163
[80,  1800] loss: 0.179
[80,  1900] loss: 0.153
[80,  2000] loss: 0.171
[80,  2100] loss: 0.171
[80,  2200] loss: 0.174
Mean Accuracy :  90.3

[87,  1300] loss: 0.156
[87,  1400] loss: 0.169
[87,  1500] loss: 0.163
Mean Accuracy :  90.85
[87,  1600] loss: 0.187
[87,  1700] loss: 0.180
[87,  1800] loss: 0.158
[87,  1900] loss: 0.175
[87,  2000] loss: 0.167
[87,  2100] loss: 0.175
[87,  2200] loss: 0.131
Mean Accuracy :  90.08
[87,  2300] loss: 0.178
[87,  2400] loss: 0.154
[87,  2500] loss: 0.158
[87,  2600] loss: 0.154
[87,  2700] loss: 0.166
[87,  2800] loss: 0.197
[87,  2900] loss: 0.149
[87,  3000] loss: 0.156
Mean Accuracy :  90.48
[87,  3100] loss: 0.161
Mean Accuracy :  90.37
[88,   100] loss: 0.160
[88,   200] loss: 0.169
[88,   300] loss: 0.154
[88,   400] loss: 0.137
[88,   500] loss: 0.147
[88,   600] loss: 0.155
[88,   700] loss: 0.185
Mean Accuracy :  90.93
[88,   800] loss: 0.169
[88,   900] loss: 0.160
[88,  1000] loss: 0.165
[88,  1100] loss: 0.144
[88,  1200] loss: 0.187
[88,  1300] loss: 0.163
[88,  1400] loss: 0.152
[88,  1500] loss: 0.187
Mean Accuracy :  90.62
[88,  1600] loss: 0.148
[88,  1700] loss: 0.16

Mean Accuracy :  90.86
[95,   800] loss: 0.151
[95,   900] loss: 0.150
[95,  1000] loss: 0.165
[95,  1100] loss: 0.149
[95,  1200] loss: 0.155
[95,  1300] loss: 0.199
[95,  1400] loss: 0.159
[95,  1500] loss: 0.161
Mean Accuracy :  90.86
[95,  1600] loss: 0.167
[95,  1700] loss: 0.162
[95,  1800] loss: 0.170
[95,  1900] loss: 0.171
[95,  2000] loss: 0.162
[95,  2100] loss: 0.158
[95,  2200] loss: 0.156
Mean Accuracy :  89.96
[95,  2300] loss: 0.179
[95,  2400] loss: 0.150
[95,  2500] loss: 0.180
[95,  2600] loss: 0.142
[95,  2700] loss: 0.190
[95,  2800] loss: 0.158
[95,  2900] loss: 0.163
[95,  3000] loss: 0.157
Mean Accuracy :  90.42
[95,  3100] loss: 0.173
Mean Accuracy :  90.42
[96,   100] loss: 0.148
[96,   200] loss: 0.143
[96,   300] loss: 0.160
[96,   400] loss: 0.165
[96,   500] loss: 0.156
[96,   600] loss: 0.153
[96,   700] loss: 0.176
Mean Accuracy :  90.84
[96,   800] loss: 0.130
[96,   900] loss: 0.170
[96,  1000] loss: 0.175
[96,  1100] loss: 0.152
[96,  1200] loss: 0.13

## Test

In [7]:
class_correct = np.zeros(10)
class_total = np.zeros(10)

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2f %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print('Mean Accuracy : ', 100*np.sum(class_correct)/np.sum(class_total))

Accuracy of plane : 92.200000 %
Accuracy of   car : 93.700000 %
Accuracy of  bird : 89.000000 %
Accuracy of   cat : 86.500000 %
Accuracy of  deer : 90.600000 %
Accuracy of   dog : 90.000000 %
Accuracy of  frog : 94.500000 %
Accuracy of horse : 92.100000 %
Accuracy of  ship : 92.700000 %
Accuracy of truck : 93.800000 %
Mean Accuracy :  91.51
